In [1]:
from html import escape

In [3]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a,2))

def html_str(s):
    return html_escape(s).replace('\n','<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item)) for items in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(k,v) for k,v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [6]:
print(html_str("""this is
a multi line string
with special characters 40 < 50"""))

this is<br/>
a multi line string<br/>
with special characters 40 &lt; 50


How to have these functions in such a way that in case we need to use them we need not use a if else condition in a function

In [11]:
def singledispatch(fn):

    registry = {}
    registry[object] = fn
    registry[int] = lambda a: '{0}(<i>{1}</i>)'.format(a,str(hex(a)))
    registry[str] = lambda s: html_escape(s).replace('\n','<br/>\n')

    def inner(arg):
        return registry.get(type(fn),registry[object])(arg) #registry object will return the function only
    
    return inner

In [12]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [13]:
htmlize('1 < 100')

'1 &lt; 100'

## Problems: the injection of new functions in the decorators isn't generic. We should be able to add new functions as and when we want and not have them hardcoded

In [43]:
def singledispatch(fn):
    #initializing a decorator
    registry = {}

    registry[object] = fn

    def decorated(arg):
        return registry.get(type(fn),registry[object])(arg)
    
    def register(type_): #making a decorator inside a decorator
        def inner(fn):
            registry[type_] = fn
            return fn
        
        return inner
    
    decorated.register = register #we are doing this so we can access register as an attribute of singledispatch 
    return decorated

In [44]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [45]:
htmlize("1 < 100")

'1 &lt; 100'

In [46]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i>)'.format(a,str(hex(a)))

In [47]:
htmlize(100)

'100'